# S1S2 / S3 Temperature Score Calculation

This notebook provides a comparison between temparature calculation for scopes S1S2 and S3

## Getting started
Make sure you are running the notebook with the requirements installed available in the example folder.

If you see errors when attempting to load the ITR modules, go to the top-level ITR directory, activate the `itr_env` conda environment (using `conda activate itr_env` and execute the command `pip install -e .`.  Then try again, or hit the <i class="fas fa-forward"></i> button above.

In [ ]:
import os
import sys
import warnings

If all is well, the list of paths below will prioritize loading from the environment established for the ITR tool.  Please contact us if not.

In [ ]:
# Load the ITR library files

import ITR
from ITR import data_dir
from ITR.data.excel import ExcelProviderProductionBenchmark, ExcelProviderIntensityBenchmark
from ITR.data.template import TemplateProviderCompany
from ITR.data.base_providers import BaseProviderProductionBenchmark, BaseProviderIntensityBenchmark
from ITR.data.data_warehouse import DataWarehouse
from ITR.portfolio_aggregation import PortfolioAggregationMethod
from ITR.temperature_score import TemperatureScore
from ITR.interfaces import ETimeFrames, EScope, IProductionBenchmarkScopes, IEIBenchmarkScopes
import pandas as pd

from ITR.data.osc_units import ureg, Q_, PA_, requantify_df_from_columns

from ITR_examples.utils import collect_company_contributions, plot_grouped_statistics, anonymize, \
        plot_grouped_heatmap, print_grouped_scores, get_contributions_per_group

In [ ]:
print("Testing unit registry\n=====================")
one_co2 = ureg("CO2e")
print(f"The gas species CO2e, which was a gwp of 1: {one_co2}")

one_Gt_co2 = ureg("Gt CO2e")
print(f"A gigaton of CO2e: {one_Gt_co2}")

## Load the production and intensity benchmarks

Prepare the various benchmark files; the OECM benchmark is the default after the next cell finishes.

In [ ]:
import json

benchmark_prod_json = os.path.join(data_dir, "benchmark_production_OECM.json")
benchmark_EI_OECM_S1S2 = os.path.join(data_dir, "benchmark_EI_OECM_PC.json")
benchmark_EI_OECM_S3 = os.path.join(data_dir, "benchmark_EI_OECM_S3.json")

# load production benchmarks
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)
prod_bms = IProductionBenchmarkScopes.model_validate(parsed_json)
base_production_bm = BaseProviderProductionBenchmark(production_benchmarks=prod_bms)

# load intensity benchmarks

# OECM for scope S1S2
with open(benchmark_EI_OECM_S1S2) as json_file:
    parsed_json = json.load(json_file)
ei_bms_s1s2 = IEIBenchmarkScopes.model_validate(parsed_json)
OECM_EI_bm_s1s2 = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms_s1s2)

# OECM for scope S3
with open(benchmark_EI_OECM_S3) as json_file:
    parsed_json = json.load(json_file)
ei_bms_s3 = IEIBenchmarkScopes.model_validate(parsed_json)
OECM_EI_bm_s3 = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms_s3)

base_intensity_bm_s1s2 = OECM_EI_bm_s1s2
base_intensity_bm_s3 = OECM_EI_bm_s3

### Logging
The ITR module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider

In [ ]:
# Remove the # and space on the next line to point the template_data_path variable at your own data
# template_data_path = "data/your_template_here.xlsx"

template_data_path = os.path.abspath('../src/ITR_examples/data/20220927 ITR V2 Sample Data.xlsx')
template_company_data = TemplateProviderCompany(excel_path=template_data_path)

## Create the Data Warehouse

In [ ]:
template_company_data._companies[0].production_metric

In [ ]:

print(f"S1S2:\nBenchmark Temperature = {base_intensity_bm_s1s2.benchmark_temperature}\n\
Benchmark Global Budget = {base_intensity_bm_s1s2.benchmark_global_budget}\n\
AFOLU included = {base_intensity_bm_s1s2.is_AFOLU_included}")

template_provider_s3 = DataWarehouse(template_company_data, base_production_bm, base_intensity_bm_s3)
print(f"S3:\nBenchmark Temperature = {base_intensity_bm_s3.benchmark_temperature}\n\
Benchmark Global Budget = {base_intensity_bm_s3.benchmark_global_budget}\n\
AFOLU included = {base_intensity_bm_s3.is_AFOLU_included}")

## Load your portfolio

The portfolio data is a sheet in the Data Template named "Portfolio".

Please see the technical documentation in the [Data Template Requirements](https://github.com/os-c/ITR/blob/main/docs/DataTemplateRequirements.rst) section for details on data requirements.

In [ ]:
df_portfolio = requantify_df_from_columns(pd.read_excel(template_data_path, sheet_name="Portfolio"))
display(df_portfolio.tail())

To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://ofbdabv.github.io/ITR/autoapi/ITR/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [ ]:
companies = ITR.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the enhanced portfolio you'll find your original portfolio, with calculated temperature scores, trajectory and target scores and overshoot/undershoot ration, and a temperature_result which is current set to zero for all valid calculations.

In [ ]:
template_provider = DataWarehouse(template_company_data, base_production_bm, base_intensity_bm_s1s2)
temperature_score_s1s2 = TemperatureScore(
    time_frames = [ETimeFrames.LONG],
    scopes=[EScope.S1S2],
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
enhanced_portfolio_s1s2 = temperature_score_s1s2.calculate(data_warehouse=template_provider, portfolio=companies)

temperature_score_s3 = TemperatureScore(
    time_frames = [ETimeFrames.LONG],
    scopes=[EScope.S3],
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
template_provider.update_benchmarks(base_production_bm, base_intensity_bm_s3)
enhanced_portfolio_s3 = temperature_score_s3.calculate(data_warehouse=template_provider, portfolio=companies)

For every company the tool assigns a score for all the requested timeframe and scope combinations.

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    display(enhanced_portfolio_s1s2[['company_name', 'time_frame', 'scope', 'temperature_score']])

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    display(enhanced_portfolio_s3[['company_name', 'time_frame', 'scope', 'temperature_score']])